In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math, scipy
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics
import time
from model.Least_Square import Least_Square
from opt_algorithms.gradient_descent import gradient_descent
from opt_algorithms.accelerated_gradient_descent import accelerated_gradient_descent
from opt_algorithms.newton_method import newton_method
from my_utils.visualization import *
#from urllib.error import HTTPError

In [6]:
s = "../data/kc_house_data.csv"
print('From local House path:', s)
df_housesales = pd.read_csv(s, encoding='utf-8')
df_housesales['price'] = df_housesales['price'].apply(lambda x: x/1e6)
df_housesales = df_housesales[df_housesales['bedrooms']!=33]
# for col in ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']:
#   df_housesales[col] = df_housesales[col].apply(lambda x: x/1e4)
df_housesales.head()

From local House path: ../data/kc_house_data.csv


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,0.2219,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,0.5380,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,0.1800,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,0.6040,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,0.5100,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [7]:
A = df_housesales[df_housesales.columns[3:]]
b = df_housesales['price'].values
ls = Least_Square(A, b)
ls.hess

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
bedrooms,24.373217,15.002500,1.501385e+04,1.043442e+05,10.255094,0.049824,1.694295,23.021763,5.241934e+01,1.279911e+04,2.214741e+03,1.330029e+04,5.835490e+02,6.613886e+05,3.207260e+02,-8.241343e+02,1.389850e+04,8.761282e+04
bathrooms,15.002500,10.136246,9.870488e+03,6.952601e+04,6.739778,0.040424,1.213535,14.302510,3.360740e+01,8.443077e+03,1.427411e+03,8.363570e+03,3.886779e+02,4.150183e+05,2.012646e+02,-5.171226e+02,9.007440e+03,5.769441e+04
sqft_living,15013.852658,9870.487961,1.034147e+07,7.601504e+07,6567.298481,47.908039,1375.508150,14115.966383,3.350549e+04,8.774053e+06,1.567420e+06,8.217876e+06,3.922519e+05,4.080520e+08,1.978959e+05,-5.084317e+05,9.217529e+06,6.231660e+07
sqft_lot,104344.244490,69526.014123,7.601504e+07,3.886015e+09,44906.279126,382.997592,11830.037970,102498.936655,2.424153e+05,6.663993e+07,9.375105e+06,5.965235e+07,2.806662e+06,2.961346e+09,1.435317e+06,-3.688137e+06,6.821154e+07,2.007733e+09
floors,10.255094,6.739778,6.567298e+03,4.490628e+04,5.047277,0.024773,0.723977,10.003797,2.346472e+01,5.812984e+03,7.543145e+02,5.905330e+03,2.551963e+02,2.930769e+05,1.421283e+02,-3.651854e+02,6.143723e+03,3.780990e+04
waterfront,0.049824,0.040424,4.790804e+01,3.829976e+02,0.024773,0.015095,0.056862,0.053343,1.324319e-01,3.733154e+01,1.057650e+01,2.962002e+01,7.734766e+00,1.480805e+03,7.175950e-01,-1.845888e+00,4.024958e+01,3.379118e+02
view,1.694295,1.213535,1.375508e+03,1.183004e+04,0.723977,0.056862,1.284497,1.643730,4.041211e+00,1.050804e+03,3.247044e+02,9.212059e+02,1.036044e+02,4.596679e+04,2.228816e+01,-5.728697e+01,1.225742e+03,9.026399e+03
condition,23.021763,14.302510,1.411597e+04,1.024989e+05,10.003797,0.053343,1.643730,24.099000,5.199926e+01,1.202655e+04,2.089415e+03,1.342743e+04,5.442669e+02,6.688431e+05,3.243334e+02,-8.334580e+02,1.346509e+04,8.689796e+04
grade,52.419337,33.607404,3.350549e+04,2.424153e+05,23.464716,0.132432,4.041211,51.999259,1.200410e+02,2.886293e+04,4.642568e+03,3.021850e+04,1.307154e+03,1.502128e+06,7.284621e+02,-1.871750e+03,3.157506e+04,2.031521e+05
sqft_above,12799.111780,8443.077445,8.774053e+06,6.663993e+07,5812.983932,37.331543,1050.803760,12026.551491,2.886293e+04,7.768726e+06,1.005327e+06,7.071408e+06,3.176278e+05,3.508275e+08,1.701347e+05,-4.371112e+05,7.936852e+06,5.444891e+07


# Gradient Descent

## Without Backtracking LineSearch

In [8]:
config = []
for alpha_ in [1e-1, 1e-2, 1e-3]:
    gd = gradient_descent(ls, alpha=alpha_)
    tmp = {
                'optimation': 'GD',
                'use_backtracking': False,
                'parameter': {
                    'alpha': str(alpha_),
                },
                'result': gd
            }
    config.append(tmp)
config_ = print_result(config)

GD finished after 17 iterations
GD finished after 19 iterations
GD finished after 22 iterations


## With Backtracking LineSearch

In [9]:
config = []
for alpha_bar in [0.1,0.5,1]:
    for ro  in [0.1, 0.5, 0.9]:
        for c in [0.1, 0.5, 0.9]:
            gd_ls = gradient_descent(ls, use_line_search=True, alpha_bar=alpha_bar, ro=ro, c=c)
            tmp = {
                'optimation': 'GD',
                'use_backtracking': True,
                'parameter': {
                    'alpha_bar': str(alpha_bar),
                    'ro': str(ro),
                    'c': str(c)
                },
                'result': gd_ls
            }
            config.append(tmp)
config_ = print_result(config)

KeyboardInterrupt: 

# Accelerated Gradient Descent

## Without Backtracking LineSearch

In [ ]:
config = []
for alpha_ in [0.1, 0.01, 0.001]:
    agd = accelerated_gradient_descent(alpha=alpha_)
    tmp = {
                'optimation': 'AGD',
                'use_backtracking': False,
                'parameter': {
                    'alpha': str(alpha_),
                },
                'result': agd
            }
    config.append(tmp)
config_ = print_result(config)

## With Backtracking LineSearch

In [ ]:
config = []
for alpha_bar in [0.1,0.5,1]:
    for ro  in [0.1, 0.5, 0.9]:
        for c in [0.1, 0.5, 0.9]:
            agd_ls = accelerated_gradient_descent(use_linesearch=True, alpha_bar=alpha_bar, ro=ro, c=c)
            tmp = {
                'optimation': 'AGD',
                'use_backtracking': True,
                'parameter': {
                    'alpha_bar': str(alpha_bar),
                    'ro': str(ro),
                    'c': str(c)
                },
                'result': agd_ls
            }
            config.append(tmp)
config_ = print_result(config)

# Newton Method

## Without Backtracking LineSearch

In [ ]:
config = []
for alpha_ in [1,
               1e-1,
               1e-2,
              ]:
    nt = newton_method(alpha=alpha_)
    tmp = {
                'optimation': 'Newton',
                'use_backtracking': False,
                'parameter': {
                    'alpha': str(alpha_),
                },
                'result': nt
            }
    config.append(tmp)
print_result(config)

## With Backtracking LineSearch

In [ ]:
config = []
for alpha_bar in [1]:
    for ro  in [0.1, 0.5, 0.9]:
        for c in [0.1, 0.5, 0.9]:
            nt_ls = newton_method(use_linesearch=True, alpha_bar=alpha_bar, ro=ro, c=c)
            tmp = {
                'optimation': 'Newton',
                'use_backtracking': True,
                'parameter': {
                    'alpha_bar': str(alpha_bar),
                    'ro': str(ro),
                    'c': str(c)
                },
                'result': nt_ls
            }
            config.append(tmp)
print_result(config)